In [7]:
import os
import pathlib
from pathlib import Path
from glob import glob
import csv
import IPython

import librosa
import pandas as pd
import torchaudio

dubbingPath = "HLKv21/애드온/한국어 더빙/"
dubbingDetailTxt = "HLKv21/애드온/한국어 더빙/sentences.txt"
subtitleDetailTxt = "HLKv21/valve/captionmod/dictionary_koreana.txt"
dubSubDictionaryPath = "HLKv21/valve/captionmod/dictionary.csv"

save_dir = "resample"
working_dir = "halfLife"

In [2]:
# download half life data
# https://bbs.ruliweb.com/news/board/1003/read/1962882
!sudo apt-get install megatools
!megadl 'https://mega.nz/#!QthmmB6L!Nr5zK9iMTtLlVLvFP5w-tLwJIQzpS-eezGMY0Y7AGDc'
!unzip -O cp949 "HLKv21.zip" -d "HLKv21/"
!unzip -O cp949 "HLKv21/애드온/한국어 더빙.zip" -d "HLKv212/애드온/한국어 더빙"


ERROR: Download failed for 'https://mega.nz/#!QthmmB6L!Nr5zK9iMTtLlVLvFP5w-tLwJIQzpS-eezGMY0Y7AGDc': Can't open local file /home/user/code/jupyter/tts/HLKv21.zip for writing: Error opening file “/home/user/code/jupyter/tts/HLKv21.zip”: File exists
Archive:  HLKv21.zip
replace HLKv21/metahook.exe? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
unzip:  cannot find or open HLKv21/애드온/한국어 더빙.zip, HLKv21/애드온/한국어 더빙.zip.zip or HLKv21/애드온/한국어 더빙.zip.ZIP.


In [2]:
# read subtitle
with open(subtitleDetailTxt, encoding="utf-16") as f:
    lines = f.readlines()

lines = [line for line in lines if line.count('"') == 4]  # line include four '"'
subtitleDict = dict()
for line in lines:
    item = line.split('"')
    itemId = item[1].strip().lower()

    sub = item[3]
    sub = sub.replace("\\n", " ")
    sub = sub.replace("…", ".")
    sub = " ".join(sub.split())  # remove duplicate space
    sub = sub.strip()
    if sub == "":
        continue  # skip empty

    subtitleDict[itemId] = sub

len(subtitleDict)

986


In [3]:
# read dub
dubPathDict = dict()

# make dict using dubbing sentences.txt
with open(dubbingDetailTxt) as f:
    lines = f.readlines()
    lines = [line for line in lines if line.count("/") == 1]

    for line in lines:
        line = line.strip().lower()
        item = line.split(" ", 1)
        itemID = item[0]
        path = item[1] + ".wav"
        if os.path.exists(dubbingPath + path):
            dubPathDict[itemID] = dubbingPath + path


# make dict using file name
dubPathList = glob(dubbingPath + "**/*.wav", recursive=True)
for item in dubPathList:
    dubPathDict[pathlib.Path(item).stem] = item  # only file name
for item in dubPathList:
    dubPathDict[item.replace(dubbingPath, "")] = item  # without basepath

In [4]:
# read dictionary
df = pd.read_csv(
    dubSubDictionaryPath,
    encoding="unicode_escape",
    usecols=[0, 1],
    names=["dubID", "subID"],
)
df = df.dropna()  # drop nan
df = df.applymap(str.lower)  # all lowercase
df = df[df["subID"].str.contains("#")]  # filter only include #
df["subID"] = df["subID"].str.replace("#", "")  # remove #

# add subText
df["subText"] = df["subID"].map(subtitleDict)
df = df.dropna()  # drop nan

# add dubPath
df["dubID"] = df["dubID"].str.replace("*", "")  # remove #
df["dubPath"] = df["dubID"].map(dubPathDict)
df["dubPathShort"] = df["dubPath"].str.replace(dubbingPath, "")  # remove #

df = df.dropna()  # drop nan
df

/home/user/anaconda3/envs/tts/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  from ipykernel import kernelapp as app


,dubID,subID,subText,dubPath,dubPathShort
2,holo/tr_holo_intro.wav,halflife_holo_intro,"안녕하십니까, 블랙 메사 재난 예방 훈련에 오신 것을 환영합니다. 이 곳에서 보호복...",HLKv21/애드온/한국어 더빙/holo/tr_holo_intro.wav,holo/tr_holo_intro.wav
3,holo/tr_holo_button.wav,halflife_holo_button,"이런 버튼이 나타나면 눌러보십시오, 그럼 제가 나와서 동작 시범을 보이겠습니다.",HLKv21/애드온/한국어 더빙/holo/tr_holo_button.wav,holo/tr_holo_button.wav
4,holo/tr_holo_move.wav,halflife_holo_move,"HEV 보호복을 입고 이동하면, 처음에는 약간 방향감각을 잃을 수도 있습니다. 잠시...",HLKv21/애드온/한국어 더빙/holo/tr_holo_move.wav,holo/tr_holo_move.wav
11,tride/c0a0_tr_gmorn.wav,halflife_c0a0_tr_goodmorn,"안녕하십니까, 블랙 메사 운송 시스템을 사용해 주셔서 감사합니다. 이 자동 기차는 ...",HLKv21/애드온/한국어 더빙/tride/c0a0_tr_gmorn.wav,tride/c0a0_tr_gmorn.wav
12,tride/c0a0_tr_time.wav,halflife_c0a0_tr_time,현재 시간은 오전 8시 47분입니다. 현재 상층 온도는 섭씨 34도입니다. 예상 최...,HLKv21/애드온/한국어 더빙/tride/c0a0_tr_time.wav,tride/c0a0_tr_time.wav
...,...,...,...,...,...
966,tr_b,tr_b,타겟 시스템 작동 중지됨.,HLKv21/애드온/한국어 더빙/sentences/tr_b.wav,sentences/tr_b.wav
967,we0,we0,경고. 고압 케이블.,HLKv21/애드온/한국어 더빙/sentences/we0.wav,sentences/we0.wav
968,we1,we1,경고. 고압 전원 케이블.,HLKv21/애드온/한국어 더빙/sentences/we1.wav,sentences/we1.wav
969,we2,we2,경고. 감전 위험.,HLKv21/애드온/한국어 더빙/sentences/we2.wav,sentences/we2.wav


In [5]:
# filter, only scientist speaker voice
df = df[df["dubPath"].str.contains("scientist")]

# remove too long wav because of out of memory
df = df[df.apply(lambda x: librosa.get_duration(filename=x["dubPath"]) < 15, axis=1)]

df

,dubID,subID,subText,dubPath,dubPathShort
186,scientist/c1a0_sci_bigday.wav,c1a0_sci_bigday,"오늘은 중요한 날이야, 프리맨.",HLKv21/애드온/한국어 더빙/scientist/c1a0_sci_bigday.wav,scientist/c1a0_sci_bigday.wav
187,scientist/c1a0_sci_getaway.wav,c1a0_sci_getaway,저리 비켜주게 프리맨. 곧 중요한 메시지가 올 거야.,HLKv21/애드온/한국어 더빙/scientist/c1a0_sci_getaway.wav,scientist/c1a0_sci_getaway.wav
188,scientist/c1a0_sci_gm.wav,c1a0_sci_gm,"안녕, 고든.",HLKv21/애드온/한국어 더빙/scientist/c1a0_sci_gm.wav,scientist/c1a0_sci_gm.wav
189,scientist/c1a0_sci_gm1.wav,c1a0_sci_gm1,"안녕, 고든.",HLKv21/애드온/한국어 더빙/scientist/c1a0_sci_gm1.wav,scientist/c1a0_sci_gm1.wav
190,scientist/c1a0_sci_itsyou.wav,c1a0_sci_itsyou,아! 자네군. 난 또 행정부서 직원인 줄 알았지. 오늘 아침에는 온통 행정 직원들만...,HLKv21/애드온/한국어 더빙/scientist/c1a0_sci_itsyou.wav,scientist/c1a0_sci_itsyou.wav
...,...,...,...,...,...
704,sc_scared0,sc_scared0,"아니, 뭐 하는 겁니까?",HLKv21/애드온/한국어 더빙/scientist/whatyoudoing.wav,scientist/whatyoudoing.wav
705,sc_scared1,sc_scared1,그만둬! 그자는 동료야!,HLKv21/애드온/한국어 더빙/scientist/stopattacking.wav,scientist/stopattacking.wav
706,sc_scared2,sc_scared2,자네 미쳤나? 그만 쏘라고!,HLKv21/애드온/한국어 더빙/scientist/youinsane.wav,scientist/youinsane.wav
707,sc_letyouin,sc_letyouin,조금만 기다리면 당신을 안으로 들어오게 하겠소.,HLKv21/애드온/한국어 더빙/scientist/letyouin.wav,scientist/letyouin.wav


In [9]:
# resample
#save resample audio
target_rate = 22050
for index, row in df.iterrows():
    savePath = os.path.join(
        save_dir, working_dir, "wavs", Path(row["dubPath"]).stem + ".wav"
    )
    speech_array, sample_rate = torchaudio.load(row["dubPath"])
    speech_array = torchaudio.functional.resample(
        speech_array, sample_rate, target_rate
    )
    os.makedirs(os.path.dirname(savePath), exist_ok=True)
    torchaudio.save(savePath, speech_array, target_rate)

#save csv
os.makedirs(os.path.dirname(savePath), exist_ok=True)
with open(
    os.path.join(save_dir, working_dir, "metadata.csv"), "w", encoding="utf-8"
) as file:
    wr = csv.writer(file, delimiter="|")
    for index, row in df.iterrows():
        lineSplit = line.split(" ", 1)
        file_name = os.path.join("wavs", Path(row["dubPath"]).stem + ".wav")
        text = row["subText"].strip()
        speaker_name = "halfLife"
        wr.writerow([file_name, text, text, speaker_name])

In [21]:
IPython.display.Audio("half_life/kss/scientist/c1a0_sci_bigday.wav")